# Data Mining / Prospecção de Dados 2022/2023

#### André Falcão DI/FCUL 2023


## Lab class TP11 - Clustering Graphs and Finding Communities


## 1. Creating Graphs and making simple representations

First let's get acquainted with darwing graphs using `graphviz`. Graphviz is a tricky library and many things can go wrong when installing it so all relevant figures are presented in the comments

So let's import this and other libraries

In [ ]:
import graphviz
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#conda install -c anaconda graphviz

First we will create a simple graph with 9 edges

In [ ]:
from graphviz import Graph

edges=[(1,3), (1,4), (3,4),
       (3,7), (4,5), (2,5),
       (2,6), (2,8), (5,6),
       (5,8), (8,6), (7,8), (1,7), (2,9)]

G = Graph()
G.engine="sfdp"
for i,j in edges: G.edge(str(i),str(j))

G.format="png"
G.render("fig_1")
G

![Figure 1](fig_1.png)


### 1.1 Create the Adjacencies and the Laplacian of a Graph

Now let's create the adjaceny matrix from the set of nodes

In [ ]:

def getGraphData(edges):
    Edges=np.array(edges)
    Nodes=np.unique(Edges)
    NN=Nodes.shape[0]

    A=np.zeros((NN,NN), dtype=int)
    for e in Edges: A[e[0]-1, e[1]-1]=1
    A+=A.T
    return Edges, Nodes, A

Edges, Nodes, A = getGraphData(edges)
print(A)

Whith this data we can make the Laplacian (L), which is just the Degree matrix (a matrix with the degree of all vertices in the corresponding diagonal element)

$$ L = D - A $$

In [ ]:
def getLaplacian(A):
    D=np.diag(A.sum(axis=1))
    return D - A

L = getLaplacian(A)
print(L)

## 2. Using Eigen structures for Spectral Graph Clustering

As we have seen, the usage of Eigen data constructed over the Laplacian is able to identify compact regions in graphs.

Let's visualise what is happening. First we will create the Eingenvalues and eigenvectors. It is clearly seen that these are not ordered

In [ ]:
evals, evecs=np.linalg.eig(L)
N=evals.shape[0]
print ("Eigenvalues")
print(np.around(evals, 3))
print ("Eigenvectors")
for i in range(N): print(np.around(evecs[:,i], 3))

In [ ]:
plt.bar(Nodes, evals)
plt.grid()
plt.show()

The Laplacian is a positive semi-definite matrix, meaning that some eigenvalues may be zero. The smallest eigenvalue corresponds to a constant value for all elements of the eigenvector

Let's get the ascending order of the eigenvalues 

In [ ]:
eord = np.argsort(evals)
print(eord)

let's plot the first and the 2nd eigenvalues

In [ ]:

L1=evecs[:,eord[0]]
plt.plot(Nodes, L1, ".--")
plt.grid()
plt.show()


In [ ]:
L2=evecs[:,eord[1]]
plt.plot(Nodes, L2, ".--")
plt.grid()
plt.show()


We can see in this last plot that the order of the of the points follows the order of the nodes in the graph. 

#### Exercise 

1. Verify that the zero line splits the graph in its most natural way
2. Identify the nodes above and below.
3. What is the most separate note? Can you understand why?

![Initial graph](fig_1.png)

We can have a different perspective if we display the points according to their order in the vector and verify that everything falls into place

In [ ]:
def plotEigenVector(LL):
    NN=LL.shape[0]
    n_order=np.argsort(LL)
    plt.plot(np.arange(NN), LL[n_order], ".--")

    for x,y in zip(np.arange(NN), LL[n_order]):
        plt.annotate(Nodes[n_order[x]], (x,y), textcoords="offset points",
                     xytext=(0,3), ha='center', size=6)
    plt.grid()
    plt.show()
    
plotEigenVector(L2)

#### Exercise
1. Check what happens when we do the same for the 3rd eigenvector. Comment your results

In [ ]:
#solution


Using the 2nd Eigenvector or separation we can have a better view of the graph.

Note that we do not need to use zero as a separation criterion. Any other value will produce different results

In [ ]:
def GraphEdgeSplitter(Nodes, edges, L2, threshold=0.0):
    
    nodes_cluster_1=set(Nodes[L2>=threshold])
    nodes_cluster_2=set(Nodes[L2< threshold])

    inner_edges_1=[]
    inner_edges_2=[]
    between_edges=[]
    for i, j in edges:
        if   i in nodes_cluster_1 and j in nodes_cluster_1: inner_edges_1.append((i,j))
        elif i in nodes_cluster_2 and j in nodes_cluster_2: inner_edges_2.append((i,j))
        else: between_edges.append((i,j))
    return inner_edges_1, inner_edges_2, between_edges
        
def plotClusteredGraph(edges_1, edges_2, edges_b):
    g = Graph('G')
    g.engine="fdp"
    with g.subgraph(name="cluster_inside1") as sg1:
        sg1.attr(label="Cluster 1")
        for i,j in edges_1: sg1.edge(str(i), str(j))

    with g.subgraph(name="cluster_inside2") as sg2:
        sg2.attr(label="Cluster 2")
        for i,j in edges_2: sg2.edge(str(i), str(j))

    for i,j in edges_b: g.edge(str(i), str(j))
    return g

e1, e2, eb = GraphEdgeSplitter(Nodes, edges, L2)
G=plotClusteredGraph(e1, e2, eb)
G.format="png"
G.render("fig_2")
G

Using -0.1 as a threshold cut

In [ ]:
e1, e2, eb = GraphEdgeSplitter(Nodes, edges, L2, threshold=-0.1 )
G = plotClusteredGraph(e1, e2, eb)
G.format="png"
G.render("fig_3")
G

### 1.3. A more complex example

Let's try it with one of the first social netwoeks analized, the karate network that has 34 nodes

In [ ]:

lines=open("soc-karate.data", "rt").readlines()
edges=[line.split() for line in lines]
edges=[(int(i),int(j)) for i, j in edges]

Edges, Nodes, A = getGraphData(edges)
L=getLaplacian(A)


And let's visualise it

In [ ]:
G = Graph()
G.engine="sfdp"
for i,j in edges: G.edge(str(i),str(j))
G.format="png"
G.render("fig_4")
G

#### Exercises 

1. Try to cluster the new graph with the methods above
    1. First identify the required steps then put them to work and plot the final result
2. Identify if you can identify other thresholds for more meaningful clustering

In [ ]:
#order of the procedure

In [ ]:
#implementation


### 1.4. Using KMeans or other common methods for graph clustering

As L2 and L3 are complementary we we can use them as projections of the graph into a new space and use the new vector coordinates to make clustering

We will use KMeans clustering for a new dataset (even though the current SKLearn seems hopelessly broken)

This new dataset has evidently 3 different communities

In [ ]:
edges=[(1,2), (1,3), (2,3),
       (3,4), (3,5), (4,6),
       (4,7), (6,7), (5,8), (5,9),
       (6,10), (7,10), (7,8), (8,9)]

edges=[(i-1, j-1) for i,j in edges]
Edges, Nodes, A = getGraphData(edges)
L=getLaplacian(A)

G = Graph()
G.engine="sfdp"
for i,j in edges: G.edge(str(i),str(j))
G.format="png"
G.render("fig_6")
G


#### Exercise

1. Check with the above methods if you can identify the different clusters for this example

In [ ]:
#solution



The usage of Kmeans is strightforward as soon as the eingenvalues are computed

In [ ]:
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

#Get 2nd and 3rd eigenvalues
eord = np.argsort(evals)
L2=evecs[:,eord[1]]
L3=evecs[:,eord[2]]

#make X matrix
X=np.stack((L2, L3)).T

#do the kmeans
k=3
kms=KMeans(k)
kms=kms.fit(X)

for i in range(k):
    print("Nodes in Cluster", i, "-->", Nodes[kms.labels_==i])


As this is in 2D we can plot the nodes in their new coordinate space

In [ ]:
plt.scatter(X[:,0], X[:,1], c=kms.labels_)
for i,(x,y) in enumerate(X):
    plt.annotate(Nodes[i], (x,y), textcoords="offset points", xytext=(0,4), ha='center', size=7)
plt.grid()
plt.show()

## 2. Identifying communities



As we have seen we may want to define directly a coomunity by attributing probabilities of each node of belonging to it. Obviously a node can belong to more than one community

we have found that we can compute the probability of two nodes being connected if they belong to a community by this expression, where $F_u$ is a factor that defines the force of a node belonging to that community

$$ P(u,v) = 1- exp(-F_u.F_v^T) $$

As we actually know thse nodes are connected (as all the other elements of the graph!), we can use this knowledge to estimate compute the best set of Fs that maximizes the likelihood of this actual graph is constructed

$$ argmax_F = \prod_{(u,v) \in E} {p(u,v)} \prod_{(u,v) \notin E} {(1- p(u,v))} $$

We can compute the log-likelihood which is actually easier to handle computationally

$$ argmax_F L(F)= \sum_{(u,v) \in E} {log(1-exp(-F_u.F_v^T))} - \sum_{(u,v) \notin E} {F_u.F_v^T} $$

This is is an actual evaluation function that for a set of weights $F_u$ is able to compute a score of the likelihood that the actual graph is produced

Let's set up a random estimation for Fs

In [ ]:

F = np.random.random((10,3))
F = (F.T/F.sum(axis=1)).T #????
print(np.around(F, 3))

As per the formula we are going to noeed the connections and the no-connections of each node

In [ ]:
def getConnections(edges, Nodes):
    NN=Nodes.shape[0]
    conns={}
    nconns={}
    for i,j in edges: 
        conns.setdefault(i, []).append(j)
        conns.setdefault(j, []).append(i)
    all_nodes = set(list(range(NN)))
    nconns = {}
    for i in Nodes: nconns[i] = list(all_nodes - set(conns[i])-{i})
    return conns, nconns

conns, nconns = getConnections(edges, Nodes)
conns, nconns

Now we can have our evaluation function of a given F matrix, wat is the likelihood of a given node $u$


In [ ]:

def evalFactor(u, F, conns, nconns):
    NN, k=F.shape
    res=0 
    for v in conns[u]: res += np.log(1-np.exp(-F[u].dot(F[v]))) 
    res -=  F[nconns[u],:].dot(F[u]).sum()
    return res

#for instance for node 1 we have a value
evalFactor(1, F, conns, nconns)


Now let's create a simple stochastic hill climbing approach that will change randomly each row of the F matrix accepting this new solution if it actually is better that what was there before

In [ ]:
def SHC(conns, nconns, F, LR=0.001, niters=1000):
    NN, k = F.shape
    res_best=[evalFactor(u, F, conns, nconns) for u in range(NN)]
    LR=0.01
    for i in range(niters):
        for u in range(NN):
            chg=np.random.random(k)*LR- LR/2
            F_old = F[u].copy()
            F[u] += chg
            F[u]=np.clip(F[u], 0, 1)
            res = evalFactor(u, F, conns, nconns)
            if res>=res_best[u]:
                res_best[u]=res
            else:
                F[u] = F_old
    res_best=[evalFactor(u, F, conns, nconns) for u in range(NN)]
    return F, sum(res_best)
    
NN=Nodes.shape[0]
k=3
F = np.zeros((NN,k)) + 0.33333
print(np.around(F, 3))
res=[evalFactor(u, F, conns, nconns) for u in range(NN)]
print("Initial Evaluation: ", sum(res))



In [ ]:
F, LL =SHC(conns, nconns, F, niters=10000, LR=0.005)

print(np.around(F, 3))
print("Final Evaluation: ", LL)

#### Exercises
1. Examine the results above and comment them comparing them to the original graph
2. Change the initalization and the learning parameters to try to get better results